# Algoritmo de localización óptima de residencias

Objetivo: Seleccionar 1000 ubicaciones de residencias en España a partir de secciones censales, maximizando la demanda potencial y evitando solapamientos espaciales.

In [1]:
import pandas as pd
import numpy as np
from math import radians
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import haversine_distances

In [2]:
path = "../data/Datos caso práctico 2025 - renta y localización (def).xlsx"
xls = pd.ExcelFile(path)
xls.sheet_names

['localizacion', 'situacion', 'renta', 'HOJA_VARIABLES', 'CHULETA_PROVINCIAS']

In [3]:
df_loc_raw = pd.read_excel(path, sheet_name="localizacion", header=None)
df_sit_raw = pd.read_excel(path, sheet_name="situacion", header=None)
df_renta_raw = pd.read_excel(path, sheet_name="renta", header=None)

df_loc_raw.head(20)

,0,1,2,3,4
0,CPRO,CMUN,Seccion,longitud,latitud
1,01,001,0100101001,-2.521144,42.813781
2,NaN,NaN,0100101002,-2.499572,42.847201
3,NaN,002,0100201001,-2.996599,42.987151
4,NaN,NaN,0100201002,-2.948063,43.034617
5,NaN,NaN,0100201003,-3.007986,43.057505
6,NaN,NaN,0100201004,-3.00251,43.060785
7,NaN,NaN,0100201005,-2.998707,43.054978
8,NaN,NaN,0100201006,-2.965457,43.065538
9,NaN,NaN,0100201007,-2.983685,43.079135


In [4]:
def find_header_row(df, keywords):
    for i in range(min(200, len(df))):
        row_text = " ".join(df.iloc[i].astype(str).fillna("").tolist()).lower()
        if all(k.lower() in row_text for k in keywords):
            return i
    return None

hdr_loc = find_header_row(df_loc_raw, ["lat", "lon"])
hdr_sit = find_header_row(df_sit_raw, ["seccion"])
hdr_renta = find_header_row(df_renta_raw, ["renta"])

hdr_loc, hdr_sit, hdr_renta

(0, None, 3)

In [5]:
df_loc = pd.read_excel(path, sheet_name="localizacion", skiprows=0)
df_sit = pd.read_excel(path, sheet_name="situacion", skiprows=7)
df_sit.head()
df_renta = pd.read_excel(path, sheet_name="renta", skiprows=7)
df_renta.head()

,,2024,2023,2024.1,2023.1,Unnamed: 5
0,01001 Alegría-Dulantzi,13086,12936,34373,33702,34037.5
1,0100101 Alegría-Dulantzi distrito 01,,,,,NaN
2,0100101001 Alegría-Dulantzi sección 01001,,,,,NaN
3,0100101002 Alegría-Dulantzi sección 01002,,,,,NaN
4,01002 Amurrio,13691,13800,33936,34421,34178.5


In [6]:
df_loc.columns
df_sit.columns
df_renta.columns

Index([' ', 2024, 2023, '2024.1', '2023.1', 'Unnamed: 5'], dtype='object')

In [7]:
df_loc   = df_loc.rename(columns={"Seccion": "id_seccion"})
df_sit   = df_sit.rename(columns={"Seccion": "id_seccion"})
df_renta = df_renta.rename(columns={"Seccion": "id_seccion"})

In [8]:
print("id_seccion" in df_loc.columns)
print("id_seccion" in df_sit.columns)
print("id_seccion" in df_renta.columns)

True
False
False


In [9]:
# MVP: trabajamos solo a nivel sección censal
df = df_loc.copy()

# Renombrar columnas clave
df = df.rename(columns={
    "Seccion": "id_seccion",
    "latitud": "lat",
    "longitud": "lon"
})

df.head()

,CPRO,CMUN,id_seccion,lon,lat
0,1.0,1.0,100101001,-2.521144,42.813781
1,NaN,NaN,100101002,-2.499572,42.847201
2,NaN,2.0,100201001,-2.996599,42.987151
3,NaN,NaN,100201002,-2.948063,43.034617
4,NaN,NaN,100201003,-3.007986,43.057505


In [10]:
# MVP mínimo: solo localización
df = df.dropna(subset=["lat", "lon"])


In [11]:
# Score trivial solo para poder ordenar (todas iguales)
df["score"] = 1.0


In [12]:
df_cand = df.sort_values("score", ascending=False).head(5000)


In [13]:
def haversine_km(p1, p2):
    return haversine_distances(
        [[radians(p1[0]), radians(p1[1])]],
        [[radians(p2[0]), radians(p2[1])]]
    )[0][0] * 6371

In [14]:
def select_locations(df, k=1000, D=3):
    selected = []
    coords_selected = []

    for _, row in df.iterrows():
        if len(selected) >= k:
            break

        point = (row["lat"], row["lon"])

        if all(haversine_km(point, c) >= D for c in coords_selected):
            selected.append(row)
            coords_selected.append(point)

    return pd.DataFrame(selected)

In [18]:
res = select_locations(df_cand, k=1000, D=3)
len(res)

1000

In [19]:
for D in [1, 2, 3, 5, 8]:
    print(D, len(select_locations(df_cand, k=1000, D=D)))

1 1000
2 1000
3 1000
5 860
8 525


In [20]:
import folium

m = folium.Map(location=[40, -3], zoom_start=6)

for _, r in res.iterrows():
    folium.CircleMarker(
        [r["lat"], r["lon"]],
        radius=3,
        color="red"
    ).add_to(m)

m